In [8]:
from pathlib import Path
import pickle
# from model_trainer.main_multitask_v3 import main

f1_cut = 0.5
count_of_bad_elms = {}

exp_dir = Path("/global/homes/d/drsmith/scratch-ml/multi_256_v27")
trials = exp_dir.glob("L3*r44095399*")
for trial in trials:
    scores_dir = trial / "scores"
    score_files = sorted(scores_dir.glob("elm_wise_f1_scores_ep*.pkl"), reverse=True)
    if len(score_files) == 0:
        continue
    final_score_file = score_files[0]
    with open(final_score_file, "rb") as f:
        final_scores = pickle.load(f)
    if not 'train' in final_scores:
        continue
    train_scores = final_scores['train']
    for elm_id, elm_f1 in train_scores.items():
        if elm_f1 < f1_cut:
            if elm_id not in count_of_bad_elms:
                count_of_bad_elms[elm_id] = 0
            count_of_bad_elms[elm_id] += 1
count_of_bad_elms = dict(sorted(count_of_bad_elms.items(), key=lambda item: item[1], reverse=True))
with open("count_of_bad_elms.pkl", "wb") as f:
    pickle.dump(count_of_bad_elms, f)